In [1]:
import pandas as pd
!pip install xlrd
import xlrd

In [2]:
data = pd.read_excel('final_data_17406.xlsx')
data.head()

,authors,category,content,date,id,img_src,section,title,topics,url,startups,mobile,apps,social,gadgets,europe,enterprise
0,Lora Kolodny,Government,Investigative reporter and co-founder of Democ...,2016-10-15,1402133,https://tctechcrunch2011.files.wordpress.com/2...,enterprise/,Multi-media journalists face jail time after r...,edward-snowden,https://techcrunch.com/2016/10/15/multi-media-...,0,0,0,1,0,0,0
1,Keith Teare,Social,\n\nThe problems associated with the widesprea...,2016-10-07,1398512,https://tctechcrunch2011.files.wordpress.com/2...,social/,The case for Twitter’s independence (unless Go...,twitter,https://techcrunch.com/2016/10/07/you-cannot-t...,0,0,0,1,0,0,0
2,Frederic Lardinois,Cloud,"OpenStack, the massive open source project tha...",2016-10-06,1398065,https://tctechcrunch2011.files.wordpress.com/2...,enterprise/,OpenStack’s latest release focuses on scalabil...,NaN,https://techcrunch.com/2016/10/06/openstacks-l...,0,0,0,0,0,0,1
3,Haje Jan Kamps,Europe,It’s been just over a year since Adobe launche...,2016-10-05,1396326,https://tctechcrunch2011.files.wordpress.com/2...,europe/,Adobe teams up with EyeEm to expand its stock ...,NaN,https://techcrunch.com/2016/10/05/adobe-eyeem-...,1,1,0,0,0,1,0
4,Dr. Nancy Markley,Startups,\n\nMany startup companies begin with either a...,2016-10-04,1396165,https://tctechcrunch2011.files.wordpress.com/2...,startups/,Due diligence is a responsibility for investor...,NaN,https://techcrunch.com/2016/10/04/due-diligenc...,1,0,0,0,0,0,0


In [4]:
df = data.drop(columns = [data.columns[0],'authors', 'section', 'url', 'category', 'date', 'id', 'img_src', 'topics'])
# data_selected
df.head()

,content,title,startups,mobile,apps,social,gadgets,europe,enterprise
0,Investigative reporter and co-founder of Democ...,Multi-media journalists face jail time after r...,0,0,0,1,0,0,0
1,\n\nThe problems associated with the widesprea...,The case for Twitter’s independence (unless Go...,0,0,0,1,0,0,0
2,"OpenStack, the massive open source project tha...",OpenStack’s latest release focuses on scalabil...,0,0,0,0,0,0,1
3,It’s been just over a year since Adobe launche...,Adobe teams up with EyeEm to expand its stock ...,1,1,0,0,0,1,0
4,\n\nMany startup companies begin with either a...,Due diligence is a responsibility for investor...,1,0,0,0,0,0,0


In [5]:
import numpy as np
df['content'].isnull().any()

True

In [7]:
df = df.dropna()
print(df.shape)
df.isnull().any()

(17386, 9)


content       False
title         False
startups      False
mobile        False
apps          False
social        False
gadgets       False
europe        False
enterprise    False
dtype: bool

In [11]:
df = df.head(1000)

In [12]:
x=df.iloc[:,0:2].values
y=df.iloc[:,2:-1].values
# using binary relevance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
x1=df.title
x2=df.content
tfidf_title=TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english').fit(x1)
X_title =  tfidf_title.transform(x1)
svd = TruncatedSVD(n_components=120)
svd.fit(X_title)
X_title = svd.transform(X_title)
df_title = pd.DataFrame(X_title, columns=['title_{}'.format(i) for i in range(120)])


tfidf_content=TfidfVectorizer(min_df=3,  max_features=10000,
        strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
        ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
        stop_words = 'english').fit(x2)
X_content =  tfidf_content.transform(x2)
svd = TruncatedSVD(n_components=500)
svd.fit(X_content)
X_content = svd.transform(X_content)
df_content = pd.DataFrame(X_content, columns=['content_{}'.format(i) for i in range(500)])
X_data = pd.concat((df_title, df_content), axis=1)
X_data.head()

MemoryError: 

In [9]:
Y_data = df.iloc[:,2:-1]
Y_data.head()

,startups,mobile,apps,social,gadgets,europe
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,0,0,0
3,1,1,0,0,0,1
4,1,0,0,0,0,0


In [10]:
# !pip install scikit-multilearn

# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
xtrain,xtest,ytrain,ytest=train_test_split(X_data,Y_data)
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(xtrain.astype(float), ytrain.astype(float))

predictions = classifier.predict(xtest.astype(float))
predictions.toarray()
from sklearn.metrics import accuracy_score
accuracy_score(ytest.astype(float),predictions)

0.15021854152288935

In [14]:
predictions.toarray()

array([[1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 1., 1., 0., 1.],
       ...,
       [1., 0., 1., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.]])